<a href="https://colab.research.google.com/github/swarathesh/Spark_ML/blob/master/Recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from pyspark.sql import SparkSession 
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
spark = SparkSession.builder.appName('recommender_system').getOrCreate()

/home/swarathesh60/.local/lib/python2.7/site-packages/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [6]:
data = spark.read.csv('movielens_ratings.csv',header=True,inferSchema=True)

In [7]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
train, test = data.randomSplit([0.9,0.1])

In [10]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [11]:
model = als.fit(train)

In [12]:
pred = model.transform(test)

In [13]:
pred.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   3.0|    14|  2.2697055|
|     85|   1.0|    12|  1.0494381|
|     65|   1.0|    22| 0.79606366|
|     65|   1.0|    16|   2.600819|
|     65|   5.0|    23|  1.3584112|
|     78|   1.0|     1|  0.9427622|
|     34|   1.0|     4|  2.1698303|
|     81|   5.0|    28|  3.3998742|
|     81|   3.0|    18|-0.35806572|
|     12|   1.0|    13|  1.5871494|
|     12|   1.0|     4|-0.47820807|
|     91|   1.0|    16|  1.3304385|
|     91|   2.0|     5|    1.70176|
|     22|   5.0|    20| 0.63853395|
|     22|   4.0|    17|  3.4802032|
|     22|   1.0|    10|  1.1316644|
|     93|   1.0|    29|  2.1166332|
|     47|   1.0|    26|-0.85830534|
|     47|   1.0|    24| -3.3724709|
|      1|   3.0|    25|  0.9748593|
+-------+------+------+-----------+
only showing top 20 rows



In [14]:
RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction').evaluate(pred)

1.5863217334635191

# test it on single user

In [25]:
user = test.filter(test['userId']==9).select(['movieId','userId'])

In [26]:
user.show()

+-------+------+------+
|movieId|userId|rating|
+-------+------+------+
|      4|     9|   1.0|
|     70|     9|   3.0|
|     83|     9|   1.0|
+-------+------+------+



In [23]:
model.transform(user).orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     70|     9| 2.6081328|
|      4|     9| 1.6831845|
|     83|     9| 1.5570519|
+-------+------+----------+

